In [1]:
import pandas as pd
import numpy as np

In [2]:
train_txt = pd.read_csv('ratings_train.txt',sep='\t')
train_txt

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [3]:
from konlpy.tag import *

In [4]:
train_txt = train_txt.dropna().reset_index(drop=True)

In [5]:
train_txt['document'] = train_txt['document'].apply(lambda x:x.replace('.',' '))

In [6]:
hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()
mecab = Mecab()
okt = Okt()

In [7]:
# 데이터에 존재하는 모든 n-grams 이용하면 차원 커지므로 
# 빈도수 카운팅으로 분석에 유의미한 n-gram 추출

from collections import defaultdict
def get_ngram_counter(docs, min_count=10):
    ngram_counter = defaultdict(int)
    for doc in docs:
        words = mecab.pos(doc, join=True) # 형태소와 품사 태그 set return
        for ngram in words:
            ngram_counter[ngram] += 1

    ngram_counter = {
        ngram:count for ngram, count in ngram_counter.items()
        if count >= min_count
    }

    return ngram_counter

ngram_counter = get_ngram_counter(train_txt.document)

In [8]:
class NgramTokenizer:

    def __init__(self, ngrams, base_tokenizer):
        self.ngrams = ngrams
        self.base_tokenizer = base_tokenizer

    def __call__(self, sent):
        return self.tokenize(sent)

    def tokenize(self, sent):
        if not sent:
            return []
        unigrams = self.base_tokenizer.pos(sent, join=True)
        return unigrams

ngram_tokenizer = NgramTokenizer(ngram_counter, mecab)

In [9]:
from tqdm import tqdm

token_list = []
for doc in tqdm(train_txt.document):
    token_list.append(ngram_tokenizer(doc))

100%|██████████| 149995/149995 [00:13<00:00, 11116.29it/s]


In [10]:
train_txt['token'] = token_list
train_txt

,id,document,label,token
0,9976970,아 더빙 진짜 짜증나네요 목소리,0,"[아/IC, 더/MAG, 빙/MAG, 진짜/MAG, 짜증/NNG, 나/NP, 네요/..."
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1,"[흠/IC, 포스터/NNP, 보고/JKB, 초딩/NNG, 영화/NNG, 줄/NNG,..."
2,10265843,너무재밓었다그래서보는것을추천한다,0,"[너무/MAG, 재/XPN, 밓었다그래서보는것을추천한다/UNKNOWN]"
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0,"[교도소/NNG, 이야기/NNG, 구먼/VCP+EF, 솔직히/MAG, 재미/NNG,..."
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,"[사이몬페그/NNP, 의/JKG, 익살/NNG, 스런/XSA+ETM, 연기/NNG,..."
...,...,...,...,...
149990,6222902,인간이 문제지 소는 뭔죄인가,0,"[인간/NNG, 이/JKS, 문제지/NNG, 소/NNG, 는/JX, 뭔/MM, 죄/..."
149991,8549745,평점이 너무 낮아서,1,"[평점/NNG, 이/JKS, 너무/MAG, 낮/VA, 아서/EC]"
149992,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0,"[이게/NP+JKS, 뭐요/NP+VCP+EF, ?/SF, 한국인/NNG, 은/JX,..."
149993,2376369,청춘 영화의 최고봉 방황과 우울했던 날들의 자화상,1,"[청춘/NNG, 영화/NNG, 의/JKG, 최고봉/NNP, 방황/NNG, 과/JC,..."


In [13]:
for item in train_txt['token'].tolist():
    if len(item) != 0:
        item = ''.join(str(e)+',' for e in item)
        item = item.replace('[','').replace(']','')
    else:
        item = ''

In [12]:
for i in range(len(train_txt['token'].tolist())):
    if len(item) != 0:
        train_txt['token'][i] = ''.join(str(e)+',' for e in train_txt['token'][i])
        train_txt['token'][i] = train_txt['token'][i].replace('[','').replace(']','')
    else:
        train_txt['token'][i] = ''

<ipython-input-12-0bd6096556dd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_txt['token'][i] = ''.join(str(e)+',' for e in train_txt['token'][i])
<ipython-input-12-0bd6096556dd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_txt['token'][i] = train_txt['token'][i].replace('[','').replace(']','')


In [14]:
train_txt['tokens_'] = train_txt['token'].apply(lambda x: ' '.join([w.split('/')[0] for w in x.split(',')]).replace('[','').replace("'",""))

In [15]:
train_txt

,id,document,label,token,tokens_
0,9976970,아 더빙 진짜 짜증나네요 목소리,0,"아/IC,더/MAG,빙/MAG,진짜/MAG,짜증/NNG,나/NP,네요/VCP+EC,...",아 더 빙 진짜 짜증 나 네요 목소리
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1,"흠/IC,포스터/NNP,보고/JKB,초딩/NNG,영화/NNG,줄/NNG,오버/NNG...",흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍 지 않 구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,"너무/MAG,재/XPN,밓었다그래서보는것을추천한다/UNKNOWN,",너무 재 밓었다그래서보는것을추천한다
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0,"교도소/NNG,이야기/NNG,구먼/VCP+EF,솔직히/MAG,재미/NNG,는/JX,...",교도소 이야기 구먼 솔직히 재미 는 없 다 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,"사이몬페그/NNP,의/JKG,익살/NNG,스런/XSA+ETM,연기/NNG,가/JKS...",사이몬페그 의 익살 스런 연기 가 돋보였 던 영화 ! 스파이더맨 에서 늙 어 보이 ...
...,...,...,...,...,...
149990,6222902,인간이 문제지 소는 뭔죄인가,0,"인간/NNG,이/JKS,문제지/NNG,소/NNG,는/JX,뭔/MM,죄/NNG,인가/...",인간 이 문제지 소 는 뭔 죄 인가
149991,8549745,평점이 너무 낮아서,1,"평점/NNG,이/JKS,너무/MAG,낮/VA,아서/EC,",평점 이 너무 낮 아서
149992,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0,"이게/NP+JKS,뭐요/NP+VCP+EF,?/SF,한국인/NNG,은/JX,거들먹거리...",이게 뭐요 ? 한국인 은 거들먹거리 고 필리핀 혼혈 은 착하 다 ?
149993,2376369,청춘 영화의 최고봉 방황과 우울했던 날들의 자화상,1,"청춘/NNG,영화/NNG,의/JKG,최고봉/NNP,방황/NNG,과/JC,우울/NNG...",청춘 영화 의 최고봉 방황 과 우울 했 던 날 들 의 자화상


In [69]:
train_df = pd.read_pickle('train_mecab_df.pkl')
train_df

,id,document,label,token,tokens_
0,9976970,아 더빙 진짜 짜증나네요 목소리,0,"아/IC,더/MAG,빙/MAG,진짜/MAG,짜증/NNG,나/NP,네요/VCP+EC,...",아 더 빙 진짜 짜증 나 네요 목소리
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1,"흠/IC,포스터/NNP,보고/JKB,초딩/NNG,영화/NNG,줄/NNG,오버/NNG...",흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍 지 않 구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,"너무/MAG,재/XPN,",너무 재
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0,"교도소/NNG,이야기/NNG,솔직히/MAG,재미/NNG,는/JX,없/VA,다/EC,...",교도소 이야기 솔직히 재미 는 없 다 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,"의/JKG,익살/NNG,스런/XSA+ETM,연기/NNG,가/JKS,돋보였/VV+EP...",의 익살 스런 연기 가 돋보였 던 영화 ! 스파이더맨 에서 늙 어 보이 기 만 했 ...
...,...,...,...,...,...
149990,6222902,인간이 문제지 소는 뭔죄인가,0,"인간/NNG,이/JKS,문제지/NNG,소/NNG,는/JX,뭔/MM,죄/NNG,인가/...",인간 이 문제지 소 는 뭔 죄 인가
149991,8549745,평점이 너무 낮아서,1,"평점/NNG,이/JKS,너무/MAG,낮/VA,아서/EC,",평점 이 너무 낮 아서
149992,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0,"이게/NP+JKS,?/SF,한국인/NNG,은/JX,고/EC,은/JX,착하/VA,다/...",이게 ? 한국인 은 고 은 착하 다 ?
149993,2376369,청춘 영화의 최고봉 방황과 우울했던 날들의 자화상,1,"청춘/NNG,영화/NNG,의/JKG,최고봉/NNP,방황/NNG,과/JC,우울/NNG...",청춘 영화 의 최고봉 방황 과 우울 했 던 날 들 의 자화상


In [16]:
tokens_list = [tokens.split() for tokens in train_txt['tokens_'].tolist()]

In [17]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

### Word2Vec 모델 생성

In [18]:
from gensim.models import Word2Vec

word2vec = Word2Vec(
    sentences = tokens_list,
    size = 300,
    min_count = 2,
    window = 10,
    sg = 1,
    hs = 1,
    iter = 10
)

In [19]:
word2vec.save('NaverMovie_mecab300.model')

In [20]:
model = Word2Vec.load('NaverMovie_mecab300.model')

In [22]:
print(model.most_similar(positive=['최고']))

[('단연', 0.5796699523925781), ('이태준', 0.5773665904998779), ('아미르칸', 0.5494681596755981), ('넘칩니다', 0.5446469783782959), ('램페이지', 0.5355502963066101), ('최악', 0.5300050377845764), ('!!~', 0.5281621217727661), ('에릭바나', 0.5179247856140137), ('으뜸', 0.5167334675788879), ('이선준', 0.5124340057373047)]


<ipython-input-22-4a2bd6f98709>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  print(model.most_similar(positive=['최고']))


In [23]:
print(model.most_similar(positive=['재미']))

[('재미없', 0.4675731658935547), ('스릴', 0.4647167921066284), ('재밌', 0.454572468996048), ('긴커녕', 0.45112064480781555), ('안배', 0.45051103830337524), ('재비', 0.4494163990020752), ('퐝당', 0.43898892402648926), ('므니다', 0.4343094229698181), ('이무', 0.4315112829208374), ('재미있', 0.43043625354766846)]


<ipython-input-23-e4d16dbef74c>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  print(model.most_similar(positive=['재미']))


In [91]:
word_embedding = word2vec.wv.__getitem__('평점')
word_embedding

array([ 1.22576393e-01, -1.04858644e-01, -2.03935633e-04,  3.95189561e-02,
       -2.67083262e-04, -2.01508224e-01, -1.29603386e-01, -1.11040622e-01,
        1.15654431e-01, -1.59261301e-02,  4.69409376e-02, -2.62131572e-01,
       -1.92012206e-01,  3.90205719e-03,  2.78855115e-01, -4.80697118e-02,
        4.22946036e-01, -8.56568068e-02, -2.13342458e-01,  2.44712353e-01,
        3.48381437e-02,  6.27905652e-02, -2.55874753e-01, -2.15082631e-01,
        1.75858945e-01, -4.67474163e-02, -5.03683627e-01,  2.56941140e-01,
       -2.09262162e-01,  9.79030281e-02,  1.61365852e-01, -3.37723523e-01,
       -2.24040538e-01,  9.87360403e-02,  4.13185731e-02, -2.89042085e-01,
       -7.87720606e-02, -1.60670877e-01, -2.72742044e-02,  4.29522768e-02,
       -2.13146970e-01, -2.56763488e-01,  2.20873058e-01,  6.57544658e-02,
        8.16274658e-02, -9.76367146e-02,  1.55808061e-01,  2.51175091e-02,
       -3.71913105e-01, -7.92512372e-02, -5.02170444e-01, -9.48391259e-02,
       -1.03436962e-01,  